In [1]:
#import necessary libraries
import requests as req
from datetime import datetime
from pytz import timezone
import pandas as pd
import bs4 as soup
import nltk
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
import pandas as pd

In [2]:
# Give the links for extraction 

url="https://www.investing.com/equities/facebook-inc-news"
url2="https://www.investing.com/equities/facebook-inc-news/2"


In [3]:
# function for extracting the news article

def newssentimentanalysis(url):
    
    ## use proxy if needed
    #proxy={"https": "46.223.255.4:3128","http": "46.223.255.4:3128"}
    
    # use the header similar to the browser
    header={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"}
    
    
    #get the page into r
    r=req.get(url,headers=header,timeout=4)
    
    #parse the page to html file
    parsed=soup.BeautifulSoup(r.content,'html.parser')
    
    #the class of the page to find the list is mentioned
    par=parsed.findAll("div",{"class":"mediumTitle1"})
    
    #the class of the file to find only the reuters articles is mentioned below, you can use external articles also
    found=par[-1].findAll("article",{"class":["js-article-item articleItem"]})#,"js-external-link-wrapper articleItem"]})
    
    # the article infos are stored in found
    # get title and link in the below for loop
    title=[]
    link=[]
    for i in range(len(found)):
        #in the found file, the articles are in "a", title class
        tmp=found[i].find("a",{"class":"title"})
        title.append(tmp["title"])
        #the link is in href so we can get it via tmp["href"]
        link.append("https://www.investing.com/"+tmp["href"])

    #now use the for loop to get the articles in the links found
    article=[]
    polarity=[]
    time=[]
    for i in range(len(link)):
        post=[]
        #load page using the header
        page=req.get(link[i],headers=header)
        parsedpage=soup.BeautifulSoup(page.content,"html.parser")
        # the article time is in contentSectionDetails
        tmptime=parsedpage.find("div",{"class":"contentSectionDetails"})
        
        #it is possible to see the time in the article as 32 minutes ago (Mar 09, 2021 01:17AM ET)
        #or we can just see it as Mar 09, 2021 01:17AM ET
        
        #so first split the string using (, if more than 1, then remove the ) and save it
        if len(tmptime.find("span").text.split("("))>1:
            time.append(tmptime.find("span").text.split("(")[-1].replace(")",""))
        #if ( is not spliting .ie., the len==1, then save the string as time
        elif len(tmptime.find("span").text.split("("))==1:
            time.append(tmptime.find("span").text)
        
        #get the article in the below class
        art=parsedpage.find("div",{"class":"WYSIWYG articlePage"})
        # the article is in many different paras, so findall paras below
        art=art.findAll("p")
        #loop through each para and get text
        for ii in range(len(art)):
            post.append(art[ii].get_text())
        #append to the article array
        article.append(post)
        #time.sleep(1) #use if needed
    #join all the paras as a single text file and pass through the vader analyser
    for j in range(len(article)):
        article[j]=" ".join(article[j])
        score=sia.polarity_scores(article[j]) #get the polarity and append
        polarity.append(score) 
    #convert the time string to datetime format. Also get the timezone localized
    tz=timezone("US/Eastern") # the articles are all in ET
    #tx.localize localizes the time
    time=[tz.localize(datetime.strptime(t, '%b %d, %Y %I:%M%p ET')) for t in time]
    final=pd.DataFrame([time,title,polarity,link,article],index=["time","title","polarity","link","full_article"])
    return final.T

In [4]:
# Scrape the data and see the polarity of articles in link 1
data=newssentimentanalysis(url)
data.head()


# Description of the DataFrame
# time: Time of release of the article
# title: Title of the article
# link: link of the respective article
# full_artile: The entire article
# polarity: it the sentiment analysis, it has 4 classes and their values range from 0 to 1
    # neg: the negitivity rating 
    # neu: neutrailty rating
    # pos: positivity rating
    # compound: the combined rating to the article
    

,time,title,polarity,link,full_article
0,2021-09-19 16:00:00-04:00,Netanyahu suggests on Facebook that Biden fell...,"{'neg': 0.047, 'neu': 0.904, 'pos': 0.049, 'co...",https://www.investing.com//news/world-news/net...,(Removes typographical error in first paragrap...
1,2021-09-19 12:41:00-04:00,Growing California wildfire spares group of gi...,"{'neg': 0.093, 'neu': 0.858, 'pos': 0.049, 'co...",https://www.investing.com//news/world-news/gro...,(Reuters) - A group of landmark giant sequoias...
2,2021-09-19 09:21:00-04:00,Thousands march for LGBTQ+ rights in Ukraine d...,"{'neg': 0.064, 'neu': 0.818, 'pos': 0.118, 'co...",https://www.investing.com//news/world-news/tho...,KYIV (Reuters) - Thousands of people including...
3,2021-09-19 05:21:00-04:00,Hong Kong opposition trade union group to disband,"{'neg': 0.092, 'neu': 0.851, 'pos': 0.057, 'co...",https://www.investing.com//news/economy/hong-k...,By Pak Yiu HONG KONG (Reuters) - A Hong Kong o...
4,2021-09-18 19:15:00-04:00,"As news of U.S. flights back to Haiti spreads,...","{'neg': 0.07, 'neu': 0.885, 'pos': 0.045, 'com...",https://www.investing.com//news/world-news/as-...,"By Alexandra Ulmer CIUDAD ACUÑA, Mexico (Reute..."


In [6]:
# Scrape the data and see the polarity of articles in link 2
data=newssentimentanalysis(url2)
data.head()


# Description of the DataFrame
# time: Time of release of the article
# title: Title of the article
# link: link of the respective article
# full_artile: The entire article
# polarity: it the sentiment analysis, it has 4 classes and their values range from 0 to 1
    # neg: the negitivity rating 
    # neu: neutrailty rating
    # pos: positivity rating
    # compound: the combined rating to the article
    

,time,title,polarity,link,full_article
0,2021-09-17 02:26:00-04:00,Martin Sorrell's S4 Capital acquires tech serv...,"{'neg': 0.008, 'neu': 0.867, 'pos': 0.125, 'co...",https://www.investing.com//news/stock-market-n...,LONDON (Reuters) -Martin Sorrell's S4 Capital ...
1,2021-09-16 22:16:00-04:00,Exclusive-Facebook cracks down on harmful real...,"{'neg': 0.106, 'neu': 0.863, 'pos': 0.031, 'co...",https://www.investing.com//news/world-news/exc...,By Elizabeth Culliford and Fanny Potkin (Reute...
